In [1]:
dataset= 'Adult'  #Name of the dataset from ['Adult','Bank']
order= 'random'  # Order of data points with respect to protected group - random or arbitrary

Ktarget = 10   #Target or desired number of clusters 


#Point 1. Capture all costs as squared euclidean 

In [2]:
import numpy as np
import math
import pandas as pd
from sklearn.utils import shuffle
import random
import numba
from numba import jit , njit
from csv import writer
import time
from scipy.spatial import distance
#!pip install scikit-learn-extra
from sklearn.cluster import KMeans
import itertools
import matplotlib.pyplot as plt

In [3]:
def load_dataset(dataset):
    
    flag= True
    if dataset=='Adult':
        _filename = 'adult_p.csv'
        OriCenters=[]
    elif dataset=='Bank':
        _filename = 'bank_p.csv'
        OriCenters=[]
        
    elif dataset=='Synthetic-1d':
        
        OriCenters = np.array([2,8,10,15])
        cluster1 = np.random.normal(OriCenters[0], 1, 1000)
        cluster2 = np.random.normal(OriCenters[1], 2, 1000)
        cluster3 = np.random.normal(OriCenters[2], 3, 1000)
        cluster4 = np.random.normal(OriCenters[3], 2, 1000)
        df = pd.DataFrame(np.concatenate((cluster1,cluster2,cluster3,cluster4)), columns=['X'])
        
        df=shuffleDataset(df)
        
        flag= False
        
        
        
    elif dataset=='Synthetic-2d':
        
        OriCenters = np.array([[3,3],[10,8],[22,5],[36,4]])
        
        cluster1 = np.random.multivariate_normal(OriCenters[0], [[1, 0], [0, 1]], 1000).T
        cluster2 = np.random.multivariate_normal(OriCenters[1], [[1, 0], [0, 1]], 1000).T
        cluster3 = np.random.multivariate_normal(OriCenters[2], [[1, 0], [0, 1]], 1000).T
        cluster4 = np.random.multivariate_normal(OriCenters[3], [[1, 0], [0, 1]], 1000).T
        df = pd.DataFrame(np.concatenate((cluster1,cluster2,cluster3,cluster4),axis=1).T, columns=['X','Y'])
        plot2Ddata(df['X'],df['Y'])
        df=shuffleDataset(df)
        flag= False
        
    elif dataset=='Synthetic-arb':
        flag= False
        pass
    
    if flag:
        
        df = pd.read_csv(_filename, sep=',')#[:10000]
        df= df.round(decimals=5)
        df = df.dropna()
    
        if order=='random':
            df=shuffleDataset(df)
        
    print('Dataset loaded........')
    
    #df.to_csv('SeededData/'+str(dataset)+'_seed_'+str(seed)+'.csv',index=False)
    
    return df,OriCenters
        
def plot2Ddata(x,y):
    plt.plot(x, y, 'x')
    plt.axis('equal')
    plt.show()

In [4]:
def shuffleDataset(df):

    df = shuffle(df)
    df = shuffle(df)    #random with respect to group and distance
    df = shuffle(df)
    df = df.reset_index()
    df = df.drop(columns='index')
    return df
    
    

In [5]:
@njit(parallel=False)
def EuclideanDistance(x,y):
    sum_ = 0
    
    for i in range(0,len(x)):
        
        sum_ = sum_ + (x[i]-y[i])**2

    return sum_#**0.5


def EuclideanDistance_(x,y):
    sum_ = 0
#     print(x)
#     print(y)
#     print()
    
    for i in range(0,len(x)):
        
        sum_ = sum_ + (x[i]-y[i])**2

    return sum_#**0.5


In [6]:
@njit(parallel=False)
def Distances(rows,center): 
    # Numpy array - center and rows
    #Finds the distance of rows to 1 center in fast parallel way
    
    #need centers in col
    rr = len(rows)
    cc = len(center) 
   
    dist=np.zeros((rr, cc)) 
    
    
    
    for indr in range(0,len(rows)):
        
        
        
        for indce in range(0,len(center)):
            
            dis = EuclideanDistance(rows[indr],center[indce])
            dist[indr][indce]= dis
            
    
    return dist
 
#@njit(parallel=False)
def sortedDistances1(centers,row): 
    # Numpy array - center and rows
    #Finds the distance of rows to 1 center in fast parallel way
    
    distOfCenters = []
    
    
    for indce in range(0,len(centers)): # each center is tuple of index, center vector 
        dis = EuclideanDistance_(row,centers[indce][1])
        distOfCenters.append(tuple((dis,centers[indce][0])))  #add distance, original center id 
        
            
    #sorted_centers = sorted(distToCenters, key=lambda x: x[0])
    distOfCenters.sort()  #inPlace sorts 
    
    
    return distOfCenters
            
   

In [7]:
def CapHeur(Ktarget,X,capacities):
    
    n_row, n_col = X.shape
    rand_indices = np.random.choice(n_row, size = Ktarget)
    maxiter=100
  
    centroids = X[rand_indices]

    
   
    for itr in range(maxiter):
        distances_to_centroids = Distances(X, centroids)#distance.cdist(X, centroids, 'sqeuclidean')
        

        
        cluster_assignment=[]
        
        capacities_iter= np.array(capacities,dtype=np.int64)#(tuple of form (center idx,capacity))
        #Deep copies it into this np.array()
        
        for idx in range(0,len(X)):
            #print(distances_to_centroids[idx])
            sorted_centerIDs = np.argsort(distances_to_centroids[idx])
            
            for idc in sorted_centerIDs:
            
                if capacities_iter[int(idc)][1] >0:
                    cluster_assignment.append(idc)
                    capacities_iter[int(idc)][1] -=1
                    break
                    
                    
        #print("Found {} assignements for iteration {}.".format(len(cluster_assignment), itr))
               
        cluster_assignment=np.array(cluster_assignment)   
        
        new_centroids=[]
        for i  in range(Ktarget):
            
            assign= np.array([X[cluster_assignment == i]])
            #print('shape of assign is ',assign)
            med = np.mean(assign,axis=1)
            #print('shape of med is ',med)
            new_centroids.append(med)
            
            
       
            
        new_centroids = np.array(new_centroids)
        new_centroids = new_centroids.reshape(Ktarget,len(X[0]))
        
            
        if np.all(centroids == new_centroids):
                break
        
        centroids = new_centroids
        #print('centroids are ',centroids)
            
    
    
    cost = []
    for i in range(Ktarget):
        cluster_data = X[cluster_assignment == i]
        distances = distance.cdist(cluster_data, [centroids[i]], 'euclidean')
        cost.append( np.sum(distances ** 2))
    
    return np.max(cost)


def LowerBoundCapHeuristic(Ktarget,dfValues):
    
    w_list = []
    seeds=[0,100,200,300,400,500,600,700,800,900]
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        
        capacities=[]
        
        for i in range(Ktarget):
            capacities.append([i,np.ceil(len(dfValues)/Ktarget)])  #(centerId,capacity)
        
        w_list.append(CapHeur(Ktarget,dfValues,capacities))
        
        
    return w_list

In [8]:
def Algo():
    return []

def LowerBoundCapSOTA(Ktarget):
    
    seeds=[0,100,200,300,400,500,600,700,800,900]
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        
        w_list = Algo()
        
    return w_list

In [9]:
#Utility functions

def CostVanillaKmeanspp(Ktarget,dfValues):
    #k-means++

    seeds=[0,100,200,300,400,500,600,700,800,900]
    cost = []
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        kmean_obj = KMeans(n_clusters=Ktarget, init='k-means++' ,random_state=seed).fit(dfValues)
        
        cost.append(kmean_obj.inertia_)    #Sum of squarred euc dis 
        
    return np.mean(cost), np.std(cost)     #average over 10 runs  
 
    
def CostVanillaKmeansList(Ktarget,dfValues):
    
    
    seeds=[0,100,200,300,400,500,600,700,800,900]
    cost = []
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        kmean_obj = KMeans(n_clusters=Ktarget, init='random' ,random_state=seed).fit(dfValues)
        
        cost.append(kmean_obj.inertia_)    #Sum of squarred euc dis 
        
        labels = kmean_obj.labels_ 
        
        cluster_centers = kmean_obj.cluster_centers_
        
        cost_seed = []
        for i in range(Ktarget):
            cluster_data = dfValues[labels == i]
            distances = distance.cdist(cluster_data, [cluster_centers[i]], 'euclidean')
            cost_seed.append( np.sum(distances ** 2))
        cost.append(np.max(cost_seed))
        
        
    
        
    
    
    return cost#, np.std(cost)    #average over 10 runs     

def CostVanillaKmeans(Ktarget,dfValues):
    
    
    seeds=[0,100,200,300,400,500,600,700,800,900]
    cost = []
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        kmean_obj = KMeans(n_clusters=Ktarget, init='random' ,random_state=seed).fit(dfValues)
        
        cost.append(kmean_obj.inertia_)    #Sum of squarred euc dis 
        
    
    
    return np.mean(cost)#, np.std(cost)    #average over 10 runs     


def CostVanillaKrandom(Ktarget,dfValues):
    
    seeds=[0,100,200,300,400,500,600,700,800,900]
    cost = []
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        
        n_row, n_col = dfValues.shape
        rand_indices = np.random.choice(n_row, size = Ktarget)
        centroids = dfValues[rand_indices]
        
        distances_to_centroids = distance.cdist(dfValues, centroids, 'sqeuclidean')
        cluster_assignment = np.argmin(distances_to_centroids, axis = 1)
        heterogeneity=0
        for i in range(Ktarget):
            cluster_data = dfValues[cluster_assignment == i]
            distances = distance.cdist(cluster_data, [centroids[i]], 'euclidean')
            heterogeneity += np.sum(distances ** 2)
        
        cost.append(heterogeneity)
    
    return np.mean(cost)#, np.std(cost)

In [10]:
def ClusteringCost(Centers,Assignments,dfValues):
    #Centers is of form -> (indx,dfValues[indx])  , Assignment is 1d bearing centerID  dfValues are indexed rows.
    
    center_map = {}
    
    for idx, c in Centers:
        center_map[idx]=c
        
    
    cost = 0
    idx = 0 
    
    for each in Assignments:
        if each !=-1:
            cost += EuclideanDistance(dfValues[idx], center_map[each])
        
        idx += 1
        
    return cost #sq Cost 
        
        
        
    
    

In [11]:
def wStarHelperHeur(Centers):
    #Centers are of form (indx,dfValues[indx])
    
    distances = []
    
    #print('len is ',len(Centers))
    for idxi in range(0,len(Centers)):
        minDis = 384082409
        
        for idx in range(0, len(Centers)):
            if idx!=idxi:
                dis = EuclideanDistance_(Centers[idxi][1],Centers[idx][1])
                print(dis)
                if dis<minDis:
                    minDis=dis
                    
        distances.append(minDis)
        
    distances = np.sort(distances)[:10]
    
    return np.mean(np.array(distances))
        
        
        
    
    
    
    

In [12]:
def nCr(n, r):
    # If r is greater than n, return 0
    if r > n:
        return 0
     
    # If r is 0 or equal to n, return 1
    if r == 0 or n == r:
        return 1
    # Initialize the logarithmic sum to 0
    res = 0
     
    # Calculate the logarithmic sum of the numerator and denominator using loop
    for i in range(r):
        # Add the logarithm of (n-i) and subtract the logarithm of (i+1)
        res += math.log(n-i) - math.log(i+1)
    # Convert logarithmic sum back to a normal number
    return round(math.exp(res))

def CouponCollectorNonUniform(capacities,Ktarget):
    
    prob = []
    sum_ = 0 
    for each in capacities:
        sum_ += each[1]
        prob.append(each[1])
    prob=np.array(prob)
    
    prob=np.round(prob/sum_,2)
  
    expec =0
    lst = [i for i in range(0,Ktarget)]
    
    for m in range(1,Ktarget+1 ):
        
        multiplier= (-1)**(m-1)
     
        
        
        #pair_order_list = itertools.combinations(lst,m)  #Contains all possible tuples of size m 
        
        #pair_order_list = [i for i in range(0,nCr(Ktarget,m))]#list(itertools.combinations(lst,m))
    
        sum_inner=(1.0*nCr(Ktarget,m))/(prob[0]*m)
      

        
        #tuple of indexes (i,j,k)
        
        
        
        a = multiplier * sum_inner
        
        expec += (a)
    
    return int(np.ceil(np.round(expec,2)))
        
    
    
        
        
    
    
    

In [13]:

params = {'legend.fontsize': '15',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'22',
          'font.weight': '500',
           'axes.linewidth':2,
          'figure.titlesize':'xx-large',
         'ytick.labelsize':'20'}

plt.rcParams.update(params)
def plotClusters(dataset,dfValues,Centers,OriCenters,runNo,algo):
    
    centers =[]
    for idx, c in Centers:
        centers.append(c)
    centers= np.array(centers)
    
    plt.plot(dfValues[:,0], dfValues[:,1],'y.',label='Datapoints')
    plt.plot(centers[:,0], centers[:,1],'r+',label='Online centers')
    
    
        
    plt.plot(OriCenters[:,0], OriCenters[:,1],'k^',markersize=5, label='Original centers')
    
    plt.axis('equal')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(dataset+': Run '+str(runNo))
    plt.legend(ncol=3)
    plt.savefig(algo+': '+dataset+'_'+str(runNo)+'.png',dpi=600)
    plt.show()
    
#ax1.legend(bbox_to_anchor=(-0.01,1.2),loc='upper left',ncol=3,fancybox=True, shadow=False)

# Fully-Online Heuristic

In [14]:
KtargetList=[2,3,5,7,10,15,20,25,30,40]

Opencenters_over_K_mean = []
Opencenters_over_K_std=[]
Cost_over_K_mean = []
Cost_over_K_std=[]






def Liberty():
    for   KtargetOri in KtargetList:
        
        if KtargetOri > 15:
            Ktarget=int(np.ceil((KtargetOri - 15)/5))
        else:
            Ktarget=KtargetOri
            
        print('K target is ' ,KtargetOri)

        seeds=[0,100,200,300,400,500,600,700,800,900]
        no_of_centers = []
        clus_cost_list=[]
        f_r_over_run = []
        
        df,OriCenters = load_dataset(dataset)
        dfValues = df.values


        #LowerBound on offline clustering over different seedValues 

        #w_list_heur = CostVanillaKmeansList(Ktarget,dfValues)



        N = len(dfValues)

        runNo =0 
        for seed in seeds:
            random.seed(seed)
            np.random.seed(seed)
            interval_update = []
            f_r_over_time=[]


            if order=='random':
                df=shuffleDataset(df)
                dfValues = df.values


            centersOpened_temp = 0 # Reset to 0 after every 3k(1+log n ) centers.

            Assignments = [] #Store in serial order index of center and store -1 if opened as center 
            CentersOpened = 0 
            Centers = []


            for indx in range(0,Ktarget+1): #first Ktarget point open
                Centers.append((indx,dfValues[indx]))
                Assignments.append(-1)
                CentersOpened+=1
                centersOpened_temp+=1

            Centers=np.array(Centers)
            CenterCost =  wStarHelperHeur(Centers) #w_list_heur[int(seed/100)]/(3*Ktarget*np.log(N)) # Center (or facility) opening cost i.e, w*/k
            #print('Center opening cost is ',CenterCost)

            #print('if > ',np.ceil(3*Ktarget*(1+np.log(N)) ))
            #start for rest of points after first one point 
            for indx in range(Ktarget+1,len(dfValues)):

                #print('centers round ',centersOpened_temp)
                if centersOpened_temp >=Ktarget:  #Reset after every 3k(1+logn) centers and double the cost for next rounds
                    CenterCost = CenterCost*10 #It takes log(n) as log(current n read)10
                    centersOpened_temp=0
                    interval_update.append(indx)
                    
                    #print('Cost updated ')



                x=dfValues[indx]
                
                f_r_over_time.append(np.round(CenterCost,2))

                #Step 1 : Find the closest center 
                distOfCenters=sortedDistances1(Centers,x)  #returns sorted (dis,centerIndex Original) list

                closest_center_index = distOfCenters[0][1]
                closest_center_dist =  distOfCenters[0][0]

                #Step 2: with p open as center and add new index and capacity and set Assignment as -1

                prob_center = np.round(closest_center_dist/CenterCost,3)
                #print('prob p is ',prob_center)
                currProb = np.round(random.random(),3)#random.uniform(0, 1),2)   
                #print('curr prob p is ',currProb)
                if currProb < prob_center:
                    #open as center 


                    Centers=np.concatenate((Centers,np.array([(CentersOpened,x)])))#Centers.append((indx,x))
                    CentersOpened+=1
                    centersOpened_temp+=1


                    Assignments.append(-1)
                else:
                    #Step 3 else with 1-p check if that index of cluster is not full , put Assignment as inde

                    Assignments.append(closest_center_index)



            #print('Total online centers are ',CentersOpened)
            no_of_centers.append(CentersOpened)

            clus_cost = np.round(ClusteringCost(Centers,Assignments,dfValues),3)
            clus_cost_list.append(clus_cost)
            algo='Liberty et al.'
            if dataset=='Synthetic-1d' or dataset=='Synthetic-2d' or dataset=='Synthetic-arb':
                plotClusters(dataset,dfValues,Centers,OriCenters,runNo)
            runNo+=1
        Opencenters_over_K_mean.append(np.mean(no_of_centers))
        Opencenters_over_K_std.append(np.std(no_of_centers))
        Cost_over_K_mean.append(np.mean(clus_cost_list))
        Cost_over_K_std.append(np.std(clus_cost_list))

        f_r_over_run.append(f_r_over_time)
        #print('interval update list ',list(interval_update))

        f_r_over_run=np.mean(f_r_over_run,axis=0)
        #print('f_r_over_run list ',list(f_r_over_run))

    print('Opencenters_over_K_mean list ', Opencenters_over_K_mean)
    print('Opencenters_over_K_std list ', Opencenters_over_K_std)
    print('Cost_over_K_mean list ', Cost_over_K_mean)
    print('Cost_over_K_std list ', Cost_over_K_std)

    return True#no_of_centers,clus_cost_list
    
Liberty()

K target is  2
Dataset loaded........
4.1398200719
2.9998032550000002
4.1398200719
2.6931516521
2.9998032550000002
2.6931516521


/home/lxc128/.local/lib/python3.6/site-packages/ipykernel_launcher.py:66: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/lxc128/.local/lib/python3.6/site-packages/ipykernel_launcher.py:104: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/lxc128/.local/lib/python3.6/site-packages/ipykernel_launcher.py:66: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when cre

1.8245260555
1.290357683
1.8245260555
3.4910821643000003
1.290357683
3.4910821643000003
3.4724250010999995
1.1535731569999998
3.4724250010999995
1.9142099791000002
1.1535731569999998
1.9142099791000002
1.7381429710999998
1.9153104143999997
1.7381429710999998
0.28469776069999997
1.9153104143999997
0.28469776069999997
0.6121497651
3.6672401961
0.6121497651
3.2265792822
3.6672401961
3.2265792822
1.4799699396000001
2.8088075872
1.4799699396000001
2.4105246592000005
2.8088075872
2.4105246592000005
3.353499116
2.3576095947000004
3.353499116
1.9903255894999998
2.3576095947000004
1.9903255894999998
3.3188393247
0.5850081461000001
3.3188393247
3.9156296900000003
0.5850081461000001
3.9156296900000003
1.3666654621
2.8373821500000003
1.3666654621
2.8999589921
2.8373821500000003
2.8999589921
1.4575705414
2.6099115219
1.4575705414
3.5244862067000002
2.6099115219
3.5244862067000002
K target is  3
Dataset loaded........
2.5954616339000003
2.1573235434
3.6939244279
2.5954616339000003
3.3236955906999994

3.9027753365000004
2.8078935852999996
2.8965577310000006
4.0553764408
2.4645291381999996
1.5527951751000002
1.7268447945999996
3.9027753365000004
3.2042967484000004
1.8386218236999998
1.4844232770999999
1.6288637101
1.5500944112
2.7638540459
2.8078935852999996
3.2042967484000004
1.4306916095000002
1.1529890245
3.8367658019000004
3.0695218494000005
0.6169314895000002
2.8965577310000006
1.8386218236999998
1.4306916095000002
0.38298944939999996
2.2508973003999992
3.1158610879
1.066486432
4.0553764408
1.4844232770999999
1.1529890245
0.38298944939999996
2.4093684514
3.5535619853000004
1.5946925944
2.4645291381999996
1.6288637101
3.8367658019000004
2.2508973003999992
2.4093684514
2.5839435995000004
3.5202168283999997
1.5527951751000002
1.5500944112
3.0695218494000005
3.1158610879
3.5535619853000004
2.5839435995000004
1.7969768995000002
1.7268447945999996
2.7638540459
0.6169314895000002
1.066486432
1.5946925944
3.5202168283999997
1.7969768995000002
3.064020319
1.1665665667999998
2.6622475832


3.9027753365000004
2.8078935852999996
2.8965577310000006
4.0553764408
2.4645291381999996
1.5527951751000002
1.7268447945999996
1.5032653531000002
0.5742709080999999
3.7689588126999993
3.9027753365000004
3.2042967484000004
1.8386218236999998
1.4844232770999999
1.6288637101
1.5500944112
2.7638540459
4.6106285138
4.1632514226
2.0316089542
2.8078935852999996
3.2042967484000004
1.4306916095000002
1.1529890245
3.8367658019000004
3.0695218494000005
0.6169314895000002
1.4898594494
3.0511653484
1.4198881202
2.8965577310000006
1.8386218236999998
1.4306916095000002
0.38298944939999996
2.2508973003999992
3.1158610879
1.066486432
3.0169760752999997
3.7763651903000004
0.6274072944999999
4.0553764408
1.4844232770999999
1.1529890245
0.38298944939999996
2.4093684514
3.5535619853000004
1.5946925944
3.2468217868999996
4.305459276700001
0.40650878989999994
2.4645291381999996
1.6288637101
3.8367658019000004
2.2508973003999992
2.4093684514
2.5839435995000004
3.5202168283999997
4.3814567641
1.926932587499999

...

3.9027753365000004
2.8078935852999996
2.8965577310000006
4.0553764408
2.4645291381999996
1.5527951751000002
1.7268447945999996
1.5032653531000002
0.5742709080999999
3.7689588126999993
1.8239577927000004
2.7581909358
2.7589671427999996
2.1412785473
1.611522172
3.9027753365000004
3.2042967484000004
1.8386218236999998
1.4844232770999999
1.6288637101
1.5500944112
2.7638540459
4.6106285138
4.1632514226
2.0316089542
4.5416565266
2.8611562714999996
3.9405257891000005
3.6361379278000006
2.5620703028999996
2.8078935852999996
3.2042967484000004
1.4306916095000002
1.1529890245
3.8367658019000004
3.0695218494000005
0.6169314895000002
1.4898594494
3.0511653484
1.4198881202
1.0460299986000001
1.8742195511000006
2.0468131889
3.196072144400001
2.419933156500001
2.8965577310000006
1.8386218236999998
1.4306916095000002
0.38298944939999996
2.2508973003999992
3.1158610879
1.066486432
3.0169760752999997
3.7763651903000004
0.6274072944999999
2.5790521783000004
3.4959931044
4.1951908528
2.5675717622999996
2.

3.0296364283
1.8107100954
3.4553512302000007
1.7994854742000002
3.0647393814
4.0277017629
1.6843126037
3.6919462838999992
3.606080376700001
2.1128744819
3.0635281179000002
2.7843918474000002
2.5267906221999996
2.7644744987000003
2.7810199675
3.0296364283
1.8238088714999998
1.7188407778999997
2.5274847207
1.0207447975000001
1.5499367222
2.6219907425999995
0.8653156672
1.301823803
2.713737375199999
2.4712445214000005
1.4549712438999998
2.9505744265000007
1.8298516502
3.1086708142
1.8107100954
1.8238088714999998
2.04467417
2.4382996554
3.5854086595999997
4.057001077100001
3.4188904752999996
1.6145050473
2.5601433153000004
1.0736426673000001
3.3248603397000003
0.5691445436000001
2.9323647394000005
1.5127087267000001
3.3136340383
3.4553512302000007
1.7188407778999997
2.04467417
4.6821097616
3.2156532646000002
1.1462689411
3.9721078311000007
1.6566462871
0.15138265169999998
2.1846949871000003
3.7945725969000006
1.4375601573999999
4.5904708288
0.44066108909999996
0.9363856495000003
1.79948547

1.4364455174
1.4333429959
1.8504262351
2.4494573870999994
1.5557246455
0.41237574860000004
1.1896118458
3.218862074
2.6631268358
2.9915457742
0.41186888909999997
0.12850040619999997
2.6192404153999997
4.2503434619
1.4220680791000002
1.4364455174
0.8277447505
1.7568935363
1.4240213097000005
2.2500551007000005
0.5904267194000001
0.11755060900000001
3.4550074712
3.6196192068
2.5060809658000003
0.6167499619000001
0.9541013747999999
3.4620926114
3.2241245371000002
1.9132825111
1.4333429959
0.8277447505
1.1064687026
1.0896913333999998
0.6192502977999998
1.4077325787000001
0.6748302385
3.7567352542999997
2.6702886230999994
2.8393251267000004
1.4596879624
1.4488505979
3.9095383829000006
4.361904255600001
0.3914731555999999
1.8504262351
1.7568935363
1.1064687026
2.3432868006000005
1.7299504384
1.9198389735
1.5458353685000001
4.919276952500001
1.6687952301
4.0666648199
1.9637771642
1.8796656905
4.8495788419000005
3.5594035516000004
1.4813354042
2.4494573870999994
1.4240213097000005
1.08969133339

K target is  20
Dataset loaded........
2.5954616339000003
2.5954616339000003
3.9027753365000004
3.9027753365000004
3.064020319
3.064020319
0.10558509820000003
0.10558509820000003
3.0296364283
3.0296364283
1.2572013467999998
1.2572013467999998
3.3086432632
3.3086432632
1.4364455174
1.4364455174
1.9670907202000003
1.9670907202000003
4.2416239741
4.2416239741
K target is  25
Dataset loaded........
2.5954616339000003
2.1573235434
2.5954616339000003
3.3236955906999994
2.1573235434
3.3236955906999994
3.9027753365000004
2.8078935852999996
3.9027753365000004
3.2042967484000004
2.8078935852999996
3.2042967484000004
3.064020319
1.1665665667999998
3.064020319
1.2946998912
1.1665665667999998
1.2946998912
0.10558509820000003
1.5497627519
0.10558509820000003
1.4370849931
1.5497627519
1.4370849931
3.0296364283
1.8107100954
3.0296364283
1.8238088714999998
1.8107100954
1.8238088714999998
1.2572013467999998
1.3868041394000001
1.2572013467999998
0.6266621245999998
1.3868041394000001
0.6266621245999998
3.

True